In [111]:
import pandas as pd
import config
import os
import sqlalchemy
import numpy as np

In [112]:
directories = [x[0] for x in os.walk('/Users/schlinkertc/Flatiron/projects/classification/txt_files')][1:]
file_names = [x[2] for x in os.walk(directories[0])][0]

tag_paths = sorted([x+'/'+file_names[0] for x in directories],reverse=True)
pre_paths = sorted([x+'/'+file_names[1] for x in directories],reverse=True)
sub_paths = sorted([x+'/'+file_names[2] for x in directories],reverse=True)
num_paths = sorted([x+'/'+file_names[4] for x in directories],reverse=True)

#Dataframes for results from 2018-2019

tags = pd.concat([pd.read_csv(x,sep='\t',
             lineterminator='\n',
             header=0,encoding='unicode_escape') for x in tag_paths[:7]
          ],axis=0
         )

presentations = pd.concat([pd.read_csv(x,sep='\t',
             lineterminator='\n',
             header=0,encoding='utf-8') for x in pre_paths[:7]
          ],axis=0
         )

numbers = pd.concat([pd.read_csv(x,sep='\t',
             lineterminator='\n',
             header=0,encoding='unicode_escape') for x in num_paths[:7]
          ],axis=0
         )

submissions = pd.concat([pd.read_csv(x,sep='\t',
             lineterminator='\n',
             header=0,encoding='utf-8') for x in sub_paths[:7]
          ],axis=0
         )


In [113]:
submissions = submissions[submissions['form']=='10-K']

In [114]:
numbers = numbers[numbers['adsh'].isin(submissions['adsh'])]
presentations = presentations[presentations['adsh'].isin(submissions['adsh'])]

In [115]:
ISnumbers = numbers.copy(deep=True)
ISnumbers = ISnumbers.join(presentations['stmt'],)
#ISnumbers = ISnumbers[ISnumbers['stmt']=='IS']

# ISdesired_values = [
#     'Revenues',
#     'GrossProfit',
#     'OperatingIncomeLoss',
#     'NetIncomeLoss',
#     'ProfitLoss',
#     'OperatingExpenses',    
# ]

ISdesired_values = ['NetIncomeLoss',
 'EarningsPerShareBasic',
 'EarningsPerShareDiluted',
 'OperatingIncomeLoss',
 'Revenues',
 'IncomeTaxExpenseBenefit',
 'CashAndCashEquivalentsAtCarryingValue',
 'StockholdersEquity',
 'GrossProfit',
 'NetCashProvidedByUsedInOperatingActivities',
 'ProfitLoss',
 'NetCashProvidedByUsedInFinancingActivities',
 'InterestExpense',
 'WeightedAverageNumberOfSharesOutstandingBasic',
 'Assets',
 'WeightedAverageNumberOfDilutedSharesOutstanding',
 'NetCashProvidedByUsedInInvestingActivities',
 'LiabilitiesAndStockholdersEquity',
 'IncomeLossFromContinuingOperationsBeforeIncomeTaxesExtraordinaryItemsNoncontrollingInterest',
 'ShareBasedCompensation',
 'RetainedEarningsAccumulatedDeficit',
 'CashAndCashEquivalentsPeriodIncreaseDecrease',
 'ComprehensiveIncomeNetOfTax',
 'OperatingExpenses',
 'EffectiveIncomeTaxRateReconciliationAtFederalStatutoryIncomeTaxRate']

ISnumbers = ISnumbers[ISnumbers['tag'].isin(ISdesired_values)]

In [117]:
ISdf = ISnumbers.merge(submissions[['adsh','name','cik','sic','form','fy']],
               how='left',
               left_on='adsh',
               right_on='adsh')

In [118]:
ISdf.set_index(['cik'],inplace=True)
ISdf.drop(columns=['name','footnote','ddate','version','form','coreg','uom','stmt'],inplace=True)

In [119]:
#ISdf = ISdf.loc[(ISdf['qtrs']==4) & (ISdf['fy'].astype('int')==2018)]

In [120]:
#ISdf.drop_duplicates(inplace=True)

In [121]:
ISdf.drop(columns=['fy','qtrs'],inplace=True)

In [123]:
piv = ISdf.reset_index().pivot_table(index=['cik','adsh'],columns='tag',values='value')

In [124]:
piv['Income/OperationsCash'] = piv['NetCashProvidedByUsedInOperatingActivities']/piv['NetIncomeLoss']
piv['Income/FinancingCash'] = piv['NetCashProvidedByUsedInFinancingActivities']/piv['NetIncomeLoss']
piv['Income/Investing'] = piv['NetCashProvidedByUsedInInvestingActivities']/piv['NetIncomeLoss']

In [141]:
df = piv[
    ['Income/OperationsCash','Income/FinancingCash','Income/Investing',
     'Assets','LiabilitiesAndStockholdersEquity','StockholdersEquity','NetIncomeLoss',
     'NetCashProvidedByUsedInOperatingActivities','NetCashProvidedByUsedInFinancingActivities',
     'NetCashProvidedByUsedInInvestingActivities'
     ]
].reset_index()

In [142]:
df = df.merge(ISdf['sic'],
        how='right',
        left_on='cik',
        right_on='cik')

In [143]:
df = df.drop_duplicates().set_index('cik')

In [144]:
df['debt_ratio'] = (df['LiabilitiesAndStockholdersEquity']-df['StockholdersEquity'])/df['Assets']

In [145]:
df.replace([np.inf, -np.inf], np.nan,inplace=True)
df.dropna(inplace=True)

In [146]:
df

,adsh,Income/OperationsCash,Income/FinancingCash,Income/Investing,Assets,LiabilitiesAndStockholdersEquity,StockholdersEquity,NetIncomeLoss,NetCashProvidedByUsedInOperatingActivities,NetCashProvidedByUsedInFinancingActivities,NetCashProvidedByUsedInInvestingActivities,sic,debt_ratio
cik,,,,,,,,,,,,,
1800,0001047469-18-000856,5.263911,1.627736,-4.241980,5.672100e+10,6.445800e+10,2.571750e+10,7.433636e+08,3.913000e+09,1.210000e+09,-3.153333e+09,2834.0,0.683001
1800,0001047469-19-000624,7.219285,-2.167270,-5.374830,6.500100e+10,7.171150e+10,3.071050e+10,6.959600e+08,5.024333e+09,-1.508333e+09,-3.740667e+09,2834.0,0.630775
1961,0001264931-19-000040,-1.955330,0.948643,3.989816,2.015174e+06,2.015174e+06,-3.441431e+06,6.289942e+05,-1.229892e+06,5.966910e+05,2.509572e+06,7372.0,2.707759
2034,0001144204-18-051414,-1.136919,-1.162120,1.879211,9.109045e+08,9.109045e+08,2.634488e+08,-5.224764e+07,5.940133e+07,6.071800e+07,-9.818433e+07,5122.0,0.710783
2098,0001026608-18-000010,3.450583,4.135294,-5.854293,1.033979e+08,1.033979e+08,4.641397e+07,1.980548e+06,6.834044e+06,8.190148e+06,-1.159471e+07,3420.0,0.551113
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1756180,0001683168-19-001042,-0.625374,0.767577,-0.063706,2.175174e+06,2.175174e+06,9.245715e+05,7.158346e+05,-4.476640e+05,5.494580e+05,-4.560267e+04,6770.0,0.574944
1756640,0001493152-19-004213,0.829373,-1.242711,0.083820,2.497500e+04,2.497500e+04,-2.227550e+04,-5.155100e+04,-4.275500e+04,6.406300e+04,-4.321000e+03,5731.0,1.891912
1757898,0001757898-19-000005,4.484040,-2.840253,-1.451481,5.136702e+09,5.136702e+09,3.191885e+09,1.077457e+08,4.831359e+08,-3.060250e+08,-1.563908e+08,3842.0,0.378612


In [147]:
df.to_csv("/Users/schlinkertc/Flatiron/projects/classification/files/data.csv",index_label='cik')